In [ ]:
!pip3 install -U spacy
!python3 -m spacy download en_core_web_sm

!pip install torch torchvision
!pip install captum

In [2]:
import spacy

import torch
import torchtext
import torchtext.data
import torch.nn as nn
import torch.nn.functional as F

from torchtext.vocab import Vocab

from captum.attr import LayerIntegratedGradients, TokenReferenceBase, visualization


## R1 Changin the English model

- I am installing the en_core_web_sm model
- so the next line gives me an error 

The original line was 



```
spacy.load('en')
```

this was changed to 

```
spacy.load('en_core_web_sm')
```




In [3]:
nlp = spacy.load('en_core_web_sm')

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
                
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)

        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [sent len, batch size]
        
        #text = text.permute(1, 0)
                
        #text = [batch size, sent len]
        
        embedded = self.embedding(text)

        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
            
        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
                
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))

        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

In [ ]:
!wget https://github.com/pytorch/captum/raw/master/tutorials/models/imdb-model-cnn.pt

In [ ]:
model = torch.load('imdb-model-cnn.pt')
model.eval()
model = model.to(device)

In [8]:
def forward_with_sigmoid(input):
    return torch.sigmoid(model(input))

## R2:  So this is interesting :) 

- The original command was giving me an error that it couldn't load the `en` model
- Remember it was giving an error when we first tried to load it at the start of the notebook
- So I checked the underlying code, and in the defination for `Field` there is a `get_tokenizer` function which loads the `en` model.
- Changed it to `en_web_core_sm`

In [10]:
TEXT = torchtext.data.Field(lower=True, 
                            tokenize='spacy') # Had to change this from the original default value of `spacy`

In [12]:
Label = torchtext.data.LabelField(dtype = torch.float)

In [15]:
train, test = torchtext.datasets.IMDB.splits(text_field=TEXT,
                                      label_field=Label,
                                      train='train',
                                      test='test',
                                      path='data/aclImdb')

In [ ]:
len(train)

In [ ]:
test, _ = test.split(split_ratio = 0.04)